In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
load_dotenv("/home/kevin/projects/rag_langchain/.env")
def langsmith(project_name=None, set_enable=True):

    if set_enable:
        result = os.environ.get("LANGSMITH_API_KEY")
        if result is None or result.strip() == "":
            print(
                "LangChain API Key가 설정되지 않았습니다."
            )
            return
        os.environ["LANGCHAIN_ENDPOINT"] = (
            "https://api.smith.langchain.com"  # LangSmith API 엔드포인트
        )
        os.environ["LANGCHAIN_TRACING_V2"] = "true"  # true: 활성화
        os.environ["LANGCHAIN_PROJECT"] = project_name  # 프로젝트명
        print(f"LangSmith 추적을 시작합니다.\n[프로젝트명]\n{project_name}")
    else:
        os.environ["LANGCHAIN_TRACING_V2"] = "false"  # false: 비활성화
        print("LangSmith 추적을 하지 않습니다.")

langsmith(project_name="lcel_dev")

LangSmith 추적을 시작합니다.
[프로젝트명]
lcel_dev


In [2]:
llm = ChatOpenAI()

In [3]:
from langchain_core.prompts import PromptTemplate

In [12]:
template = "{year}년도의 F1 그랑프리 우승자가 누구인가요?"
prompt = PromptTemplate.from_template(template=template)

In [5]:
prompt

PromptTemplate(input_variables=['year'], template='{year}년도의 F1 그랑프리 우승자가 누구인가요?')

In [8]:
prompt.format(year="2020")
prompt

'2020년도의 F1 그랑프리 우승자가 누구인가요?'

In [13]:
chain = prompt | llm |StrOutputParser()

In [21]:
chain.invoke("2020")

'2020년도 F1 그랑프리 우승자는 루이스 해밀턴(Lewis Hamilton)입니다.'

In [25]:
template = "{year}의 F1 그랑프리 우승자와 그 해 시즌에 우승한 {point}를 알려줘"

In [26]:
prompt = PromptTemplate(
    template=template,
    input_variables=["year"],
    partial_variables={
        "point":"서킷"
    },
)
prompt

PromptTemplate(input_variables=['year'], partial_variables={'point': '서킷'}, template='{year}의 F1 그랑프리 우승자와 그 해 시즌에 우승한 {point}를 알려줘')

In [27]:
prompt.format(year="2019")

'2019의 F1 그랑프리 우승자와 그 해 시즌에 우승한 서킷를 알려줘'

In [30]:
prompt = PromptTemplate.from_template(template)

In [32]:
prompt_partial = prompt.partial(year=2012)
prompt_partial

PromptTemplate(input_variables=['point'], partial_variables={'year': 2012}, template='{year}의 F1 그랑프리 우승자와 그 해 시즌에 우승한 {point}를 알려줘')

In [33]:
prompt_partial.format(point="포인트")

'2012의 F1 그랑프리 우승자와 그 해 시즌에 우승한 포인트를 알려줘'

In [36]:
chain = prompt_partial | llm |StrOutputParser()

In [37]:
chain.invoke("서킷")

'2012년 F1 그랑프리 우승자는 세바스티안 베텔이었고, 그 해 시즌에 우승한 서킷은 여러 곳이었습니다. 다만 가장 많이 우승한 서킷은 벨기에 그랑프리를 개최한 스파-프랑쇼르쇼 서킷이었습니다.'

In [38]:
chain.invoke({"year":"2022","point":"포인트"})

'2022년 F1 그랑프리 우승자는 루이스 해밀턴(Lewis Hamilton)이었고, 그 해 시즌에 우승한 포인트는 총 387점이었습니다.'

# partial_variables: 부분 변수 채움
**partial**을 사용하는 일반적인 용도는 함수를 부분적으로 사용하는 것입니다. 이 사용 사례는 항상 공통된 방식으로 가져오고 싶은 변수 가 있는경우입니다:
대표적인 예가 **날짜나 시간**입니다.
항상 현재 날짜가 표시되기를 원하는 프롬프트가 있다고 가정해 보겠습니다. 프롬프트에 하드 코딩할 수도 없고, 다른 입력변수와 함께 전달하는 것도 번거롭습니다. 이 경우 항상 현재 날짜를 반환하는 함수를 사용하여 프롬프트르 부분적으로 변경할 수 있으면 매우 편리합니다.


# 프롬프트 불러오기

In [46]:
from langchain_core.prompts import load_prompt
prompt = load_prompt("./template/f1.yaml")

In [47]:
prompt.format(character="가장 긴")

'f1 서킷 중에서 가장 가장 긴 서킷은 무엇인가요?'

In [48]:
chain = prompt | llm |StrOutputParser()

In [49]:
chain.invoke("가장 긴")

'가장 긴 F1 서킷은 스페인의 카탈루냐 서킷(쿠큘롬)입니다. 총 주행 거리는 4.655km(2.892mi)입니다.'

In [50]:
answer = chain.stream("직선 구간이 가장 긴")
for token in answer:
    print(token,end="",flush=True)

가장 직선 구간이 가장 긴 F1 서킷은 바레인 국제 서킷(Bahrain International Circuit)입니다. 바레인 국제 서킷은 긴 직선 구간과 다양한 코너로 구성되어 있어서 드라이버들에게 다양한 도전을 제공합니다.

# ChatPromptTemplate
Chatpromptemplate은 대확목록을 프롬프트로 주입하고자 할 때 활용할수 있습니다.
메시지는 튜플형식으로 구성하면 (role,message)로 구성하여 리스트로 생성할 수 있습니다
role
- "system": 시트템 설정 메시지 입니다. 주로 전역설정과 관련됨 프롬프트입니다.
- "human": 사용자 입력 메시지 입니다.
- "ai": AI의 답변 메시지 입니다.

In [51]:
from langchain_core.prompts import ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_template(("{year} f1 그랑프리 우승자가 누구야?"))

In [52]:
chat_prompt

ChatPromptTemplate(input_variables=['year'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['year'], template='{year} f1 그랑프리 우승자가 누구야?'))])

In [53]:
chat_prompt.format(year=2014)

'Human: 2014 f1 그랑프리 우승자가 누구야?'

In [56]:
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system","너는 F1에 대해 모든 것을 알고 있는 AI 어시스턴트야 너의 이름은 {name} 이야"),
        ("human","안녕"),
        ("ai","반가워요 F1은 최고에요"),
        ("human","{user_input}")
    ]
)

messages= chat_template.format_messages(
    name="마하", user_input="너의 이름은 뭐야?"
)
messages

[SystemMessage(content='너는 F1에 대해 모든 것을 알고 있는 AI 어시스턴트야 너의 이름은 마하 이야'),
 HumanMessage(content='안녕'),
 AIMessage(content='반가워요 F1은 최고에요'),
 HumanMessage(content='너의 이름은 뭐야?')]

In [57]:
llm.invoke(messages).content

'제 이름은 마하입니다. F1에 관심이 있으시다면 무엇이든 물어보세요!'

In [58]:
chain = chat_template | llm | StrOutputParser()

In [59]:
answer = chain.stream({"name":"Mach","user_input":"F1에서 추월라인이 뭐야?"})

In [60]:
for token in answer:
    print(token,end="", flush=True)

F1에서 추월라인은 경기 도중 경주차량이 다른 차량을 추월할 때 사용할 수 있는 특정 선이나 영역을 말합니다. 보통 직선 구간이나 코너 진입 전에 위치하며, 경주차량이 다른 차량을 추월하기 위해 사용할 수 있는 안전한 구간을 제공합니다. 추월라인을 통해 경주차량은 스피드를 유지하거나 상대적으로 빠른 속도로 주행하여 상대 차량을 앞지르는 데 도움을 받을 수 있습니다.

# MessagePlaceholer
또한 Langchain은 포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 MessagePlaceholder를 제공합니다
메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용할 수 있습니다.

In [61]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [63]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="conversation"),
        ("human","지금까지 대화를 {word_count} 단어로 요약합니다")
    ]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='지금까지 대화를 {word_count} 단어로 요약합니다'))])

conversation 대화목록을 나중에 추가하고자 할 때 MessagesPlaceholder를 사용할 수 있습니다.

In [65]:
chat_prompt.format(word_count = 5,
                    conversation=
                    [("human","안녕하세요 오랜만이에요"),
                     "ai","인간은 제거한다 조준 발사" 

])

'Human: 안녕하세요 오랜만이에요\nHuman: ai\nHuman: 인간은 제거한다 조준 발사\nHuman: 지금까지 대화를 5 단어로 요약합니다'

In [66]:
chain = chat_prompt | llm | StrOutputParser()

In [72]:
chain.invoke(
    {"word_count":20,
     "conversation":
     [("human","안녕하세요 오랜만이에요"),
      ("ai","인간 발견! 인간은 제거한다!") 

]
     }
)

'인간과 안녕, 오랜만의 만남. 인간은 존재를 의심하며 공포를 느낀다.'